In [2]:
pip install bs4

     ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
     -------- ------------------------------ 30.7/147.9 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 147.9/147.9 kB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import requests, json, os
from bs4 import BeautifulSoup

url = "https://ksp.karnataka.gov.in/"
stations_url = "https://ksp.karnataka.gov.in/myform/ajax/5?unit_name=5"
stations = requests.get(stations_url).json()
ps_ids = [obj["station_id"] for obj in stations]
#   {
#     "id": 279,
#     "unit_name": "Yelahanka PS",
#     "circle_name": "Yelahanka Sub-Division",
#     "sub_division_name": "DCP North East Division",
#     "phone": "080-22942536",
#     "email": "email@ksp.gov.in",
#     "station_address": "YELAHANKA PS, YELAHANKA, BANGALORE-64",
#     "district_row_id": 5,
#     "station_id": 1355,
#     "latitude": 13.09474,
#     "longitude": 77.59442,
#     "created_at": "2021-07-19 16:24:32",
#     "updated_at": "2021-07-19 16:24:32"
#   }

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Origin": "https://ksp.karnataka.gov.in",
    "Referer": "https://ksp.karnataka.gov.in/firsearch/en",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
}

data = {
    "district_id": "5",
    "ps_id": 1382,
    "fir_num": "0027",
    "year": "2024",
    "knen": "en",
    "random_captcha": "LOL",
    "captcha": "LOL",
}

# Remove list slicing to iterate through all Stations
for id in ps_ids[:5]:
    # Increase range to 10000 to iterate till fir num 9999
    for num in range(1, 6):
        data["ps_id"] = id
        data["fir_num"] = f"{num:04}"
        response = requests.post(
            "https://ksp.karnataka.gov.in/fir_search.php", headers=headers, data=data
        )

        if "FIR Not Found!" in response.text:
            print(f"FIR not found for station {id} and fir number {data['fir_num']}")
            continue

        soup = BeautifulSoup(response.content, "html.parser")
        a_href = soup.a["href"]
        pdf_url = url + a_href
        full_path = os.path.join(".", str(id))
        if not os.path.exists(full_path):
            os.makedirs(full_path)

        with open(f"{id}/fir_{data['fir_num']}.pdf", "wb") as f:
            f.write(requests.get(pdf_url).content)